In [1]:
from urllib.request import urlretrieve
import os
import numpy as np
import tensorflow as tf
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import pandas as pd
import glob
import cv2
from scipy import signal
from numpy import linalg as LA
from scipy import ndimage
import random
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.font_manager import FontProperties

def load_img(img_path):
    # read img and fft 
    img   = cv2.imread(img_path,0)
    img   = cv2.resize(img, (256, 256), interpolation=cv2.INTER_CUBIC)
    f_1   = np.fft.fft2(img)
    fshift_1 = np.fft.fftshift(f_1)
    magnitude_spectrum_1  = np.log(np.abs(fshift_1))
    
    magnitude_spectrum_1 = magnitude_spectrum_1.copy()
    magnitude_spectrum_1  = cv2.normalize(magnitude_spectrum_1,  magnitude_spectrum_1, 0, 255, cv2.NORM_MINMAX)
    #ret,magnitude_spectrum_1 = cv2.threshold(magnitude_spectrum_1,150,255,cv2.THRESH_BINARY) #--------------------------------------- 解掉註解能夠二值化
#     4-6沒有二值化 7-9有二值化(127) 10-12(200) 13-15(150)
    plt.imshow(magnitude_spectrum_1,cmap='gray') #--------------------------------------- 解掉註解能夠看最後一張load 進來的圖長怎樣
    return magnitude_spectrum_1

def load_data():
    imgs   = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    ra_info = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    ra_dataframe  = pd.read_csv('./rough_data.csv', header=None)
    
    for k in imgs.keys():
        dir = './' + k + '/'
        for file in os.listdir(dir):
            
            if not file.lower().endswith('.jpg'):
                continue
            try:
                num,right  = file.split("_",1)
                dire,right = right.split(".",1)
                row        = 0
                col        = 0
                if dire == 'left':
                    col = 0
                else:
                    col = 1
                row = int(num) - 1
                ra_value = ra_dataframe.iloc[row,col]
                resized_img = load_img(os.path.join(dir, file))
            except OSError:
                continue
            imgs[k].append(resized_img)
            ra_info[k].append(ra_value)
    return ra_info,imgs        

def conv_img(img,filter_design):
    # apply convolve2d , return conv_array and std axis
    conv_array = signal.convolve2d(img, filter_design , mode='valid')
    x_axis_array,y_axis_array = np.where(conv_array > (np.mean(conv_array)+(np.max(conv_array)-np.mean(conv_array))/2))
    #(center_x,center_y) = ndimage.measurements.center_of_mass(conv_array)
    return conv_array,[np.std(x_axis_array),np.std(y_axis_array)]

def conv_iter(img_dic,filt):
    result_conv = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    axis  = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    for k in img_dic.keys():
        for img in img_dic[k]:
            conv_array,axis_value = conv_img(img ,filt)
            result_conv[k].append(conv_array)
            axis[k].append(axis_value)
    return result_conv,axis

def oppent_display(axes,axis):
    opponent_sum = 0
    # rough1.0-1.5
    mean_x = np.mean(np.array(axis['rough1.0-1.5'])[:,0])
    mean_y = np.mean(np.array(axis['rough1.0-1.5'])[:,1])
    # * 1 std
    width  = np.std(np.array(axis['rough1.0-1.5'])[:,0])*2*2
    height = np.std(np.array(axis['rough1.0-1.5'])[:,1])*2*2
    left_x = mean_x - width/2
    bottom_y = mean_y - height/2
    ac     = Rectangle((left_x, bottom_y), width, height,facecolor = 'None')
    for element in np.array(axis['rough1.5-2.5']):
        if ac.contains_point(element):
            opponent_sum += 1
    re     = Rectangle((left_x, bottom_y), width, height,facecolor = 'None',edgecolor='blue',linewidth = 1)
    axes.add_patch(re)
    
    # rough1.5-2.5
    mean_x = np.mean(np.array(axis['rough1.5-2.5'])[:,0])
    mean_y = np.mean(np.array(axis['rough1.5-2.5'])[:,1])
    # * 1 std
    width  = np.std(np.array(axis['rough1.5-2.5'])[:,0])*2*2
    height = np.std(np.array(axis['rough1.5-2.5'])[:,1])*2*2
    left_x = mean_x - width/2
    bottom_y = mean_y - height/2
    ac     = Rectangle((left_x, bottom_y), width, height,facecolor = 'None')
    for element in np.array(axis['rough1.0-1.5']):
        if ac.contains_point(element):
            opponent_sum += 1
    re     = Rectangle((left_x, bottom_y), width, height,facecolor = 'None',edgecolor='red',linewidth = 1)
    axes.add_patch(re)
    return opponent_sum

if __name__ == '__main__':
    color   = {'rough1.0-1.5': 'b' , 'rough1.5-2.5': 'r'}
    plt_store_x = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    plt_store_y = {'rough1.0-1.5': [] , 'rough1.5-2.5': []}
    plt_label   = {'rough1.0-1.5':'1.0μm ~ 1.6μm' , 'rough1.5-2.5' : 'Greater than 1.6μm'}
    test_filter = [[0, 0, 0], [0, 0, 1], [1, 0, 1]]
    
    # define iterate number
    iteration_num = 127
    
    # load image and return spectrum
    ra_info,img_dic = load_data()
    
    df = pd.DataFrame([], columns=['X','Y'])
    df_ans = []

    
    for i in range(iteration_num):
        img_dic,axis = conv_iter(img_dic,test_filter)
        fig, axes = plt.subplots(1, 1, figsize=(6, 6),dpi=100)
        axes.grid(linewidth=0.5)
        axes.set_axisbelow(True)
        opponent_sum = oppent_display(axes,axis)
        for k in axis.keys(): 
            axes.scatter(np.array(axis[k])[:,0],np.array(axis[k])[:,1],c=color[k],label=plt_label[k],alpha=0.5)
            
            if i == 78:
                dummy_df = pd.DataFrame(np.array(axis[k]), columns=['X','Y'])
                df = df.append(dummy_df, ignore_index=True)
                
            for index, element in enumerate(np.array(axis[k])[:,0]):
                #axes.text(np.array(axis[k])[:,0][index],np.array(axis[k])[:,1][index],"%.2f" %ra_info[k][index],fontsize=9)
                if i == 78:
                    df_ans.append(ra_info[k][index])
        
        title = 'conv_' + str(i) + '_opp:' + str(opponent_sum)
        axes.legend()
        axes.set_title(title)
        axes.set_xlabel('Feature 1(X axis)')
        axes.set_ylabel('Feature 2(Y axis)')
        plt.grid(True)
        os.makedirs('./orgfile/explosion18/', exist_ok=True)  #--------------------------------------- 要改成你想要的資料夾
        fig.savefig('./orgfile/explosion18/'+title + '.jpg') #--------------------------------------- 要改成你想要的資料夾
        plt.close(fig)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the vali

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-9d19dce9a8de>", line 131, in <module>
    img_dic,axis = conv_iter(img_dic,test_filter)
  File "<ipython-input-1-9d19dce9a8de>", line 75, in conv_iter
    conv_array,axis_value = conv_img(img ,filt)
  File "<ipython-input-1-9d19dce9a8de>", line 65, in conv_img
    conv_array = signal.convolve2d(img, filter_design , mode='valid')
  File "/opt/conda/lib/python3.6/site-packages/scipy/signal/signaltools.py", line 1025, in convolve2d
    raise ValueError('convolve2d inputs must both be 2D arrays')
ValueError: convolve2d inputs must both be 2D arrays

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
    stb = value

ValueError: convolve2d inputs must both be 2D arrays

In [20]:
export_csv = df.to_csv (r'./export_dataframe.csv', index = None, header=True)

In [2]:
dat1 = pd.DataFrame({'A': df_ans})
df = df.join(dat1)

In [3]:
print(df)
export_csv = df.to_csv (r'./export_dataframe.csv', index = None, header=True)

           X          Y      A
0   5.567873  22.515056  1.407
1   5.050969  22.133170  1.377
2   5.429778  22.188575  1.419
3   5.242599  22.149824  1.459
4   5.459888  21.482498  1.156
5   5.212256  19.946591  1.484
6   6.349991  19.891242  1.120
7   5.415653  20.722584  1.293
8   5.514069  21.739353  1.473
9   5.071436  21.381721  1.426
10  5.119621  22.442215  1.334
11  4.425290  23.588325  1.645
12  4.419199  24.639483  2.079
13  4.774743  24.524879  2.088
14  3.599356  22.809664  2.089
15  4.345460  23.790370  2.142
16  4.261135  22.525974  2.070
17  4.783387  25.068915  1.877
18  5.078925  24.042733  1.662
19  4.421677  21.707520  2.309
20  5.417709  23.520864  1.694
21  4.555735  23.002857  1.661
22  4.860141  23.912004  2.015
23  4.152009  22.848972  1.843
24  5.888740  23.170550  2.099
25  6.535983  22.729083  2.374
26  3.993911  23.825182  2.589
27  4.476272  24.991699  2.184
28  4.113908  22.614839  1.894
29  4.317136  25.125170  2.228
30  4.573783  24.242542  1.656
31  4.48